In [1]:
%matplotlib qt4

In [2]:
%load_ext autoreload
%autoreload 2
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
from hyperspy.zeta import *

In [2]:
def bin_ndarray(ndarray, new_shape, operation='sum'):
    """
    Bins an ndarray in all axes based on the target shape, by summing or
        averaging.

    Number of output dimensions must match number of input dimensions and 
        new axes must divide old ones.

    Example
    -------
    >>> m = np.arange(0,100,1).reshape((10,10))
    >>> n = bin_ndarray(m, new_shape=(5,5), operation='sum')
    >>> print(n)

    [[ 22  30  38  46  54]
     [102 110 118 126 134]
     [182 190 198 206 214]
     [262 270 278 286 294]
     [342 350 358 366 374]]

    """
    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray


In [11]:
thickness_map=hs.load('../Data/unheated/Thickness Map.dm3')
# thickness_map.plot()
l = 104 # From Andreas
tuh = thickness_map.isig[100.:200.,70.:190.].data.mean()*l

In [13]:
thickness_map.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── acquisition_mode = TEM
│       ├── beam_current = 0.0
│       ├── beam_energy = 200.0
│       ├── magnification = 8000.0
│       └── microscope = JEM-ARM200F
├── General
│   ├── date = 2016-09-12
│   ├── original_filename = Thickness Map.dm3
│   ├── time = 11:26:50
│   └── title = Thickness Map
├── Sample
│   └── description = DWNC
└── Signal
    ├── Noise_properties
    │   └── Variance_linear_model
    │       ├── gain_factor = 1.0
    │       └── gain_offset = 0.0
    ├── binned = False
    ├── quantity = Intensity (mfp)
    └── signal_type = EELS

In [4]:
thickness_map_heated=hs.load('../Data/160506_SC239_X/Thickness Map.dm3')
# thickness_map_heated.plot()
l=104
th=thickness_map_heated.isig[86.:185.,71.:151.].data.mean()*l

## Set zeta values

In [3]:
# Zeta values
GaK=[486,608]
GaL=[512,826]
AsK=[576,706]
AsL=[600,955]
GeK=[729,740,745]
GeL=[898,901,933]
PdL=[1249,1284,1316]
AuL=[3422,3426]
AuM=[2396,2387]

# Background windows
bwGeL=[.6,.7,1.4,1.5]
bwAuM=[1.88,1.96,3.7,3.9]
bwAuL=[7.3,7.5,10.6,10.8]
bwGaK=[ 9.79,9.99, 11.09, 11.29]
bwAsK=[ 8.47,8.66,9.67,9.86]
bwGeK=[9.58,9.63,10.3,10.4]
# bwPdL= [2.2,2.4,3.7,3.9]

bwGeneral=[7.3,7.5,12.2,12.4]
bwGaAsGeL=[.6,.85,1.5,1.6]
bwPdLAuM=bwAuM

#K-factors
kAuM=3.868
kGeK=2.617
kPdL=3.825
kGaK=2.492
kGeL=4.513
kAsK=2.835
kGaL=4.719
kAsL=4.315

## Test the background windows

In [4]:
s_test = hs.load('../Data/160506_SC239_X/C_EDS Spectrum Image GaAs.hdf5').sum('x').sum('y')
all_lines=['Ga_Ka','Ga_La','As_Ka','As_La','Ge_Ka','Ge_La','Pd_La','Au_Ma','Au_La']
all_bw = [bwGeneral,bwGaAsGeL,bwGeneral,bwGaAsGeL,bwGeneral,bwGaAsGeL,bwPdLAuM,bwPdLAuM,bwGeneral]
s_test.set_lines(all_lines)
s_test.plot(background_windows=all_bw,integration_windows=1.2)
# s_test.metadata

C:\Users\Johannes\AppData\Local\Continuum\Anaconda3\lib\site-packages\matplotlib\__init__.py:898: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))



## Quantify area A and B in the unheated sample

In [9]:
from hyperspy import zeta_abscorr as zabs
sA=hs.load('../Data/unheated/A_EDS Spectrum Image_60-180.hdf5')
sB=hs.load('../Data/unheated/B_EDS Spectrum Image.hdf5')

slist=[sA,sB]
snames=['Sa','Sb']
probe_current= [(0.1769+.1747)/2, .1747]
IW=1.2
time=0.01

zetaA = [AsK[1],GaK[1],GeK,PdL]
zetaB=[AuM,GeL,PdL]
zetaA = [np.mean(z) for z in zetaA] #Use mean values
zetaB = [np.mean(z) for z in zetaB] #Use mean values
zeta=[zetaA,zetaB]
elements=[['As','Ga','Ge','Pd'],['Au','Ge','Pd']]
kfactors=[[kAsK,kGaK,kGeK,kPdL],[kAuM,kGeL,kPdL]]
lines=[['As_Ka','Ga_Ka','Ge_Ka','Pd_La'],['Au_Ma','Ge_La','Pd_La']]
bw= [[bwGeneral,bwGeneral,bwGeneral,bwPdLAuM],[bwPdLAuM,bwGaAsGeL,bwPdLAuM]]
thickness= tuh
# s_ints=np.zeros(len(slist))
# plt.close('all')
s_comp_linescan_abslist = []
s_comp_linescan_zetalist = []
s_comp_linescan_cllist = []

for i in range(len(slist)):
    print(snames[i])
    s=slist[i]
    s.set_elements(elements[i])
    s.set_lines(lines[i])
    
    s.metadata.General.title = snames[i]
#     s.sum('x').sum('y').plot(background_windows=bw)
    
    s.set_microscope_parameters(real_time=time,beam_current=probe_current[i])

    s_ints = s.get_lines_intensity(integration_windows=IW,background_windows=bw[i], only_lines=lines[i])
    print('!')
    s_comp_zeta, s_pt = s.quantification(s_ints, 'zeta', zeta[i])
    print('!!')
    s_comp_cl = s.quantification(s_ints, 'CL', kfactors[i])
    print('!!!')
    s_comp_abs, s_pt = zabs.quantification_zeta(s, s_ints, zeta[i], convergence_precision=0.001)
    print('!!!!')

    s_comp_linescan_abs = [c.mean('x') for c in s_comp_abs]
    s_comp_linescan_zeta = [c.mean('x') for c in s_comp_zeta]
    s_comp_linescan_cl = [c.mean('x') for c in s_comp_cl]
    
    s_comp_linescan_abslist.append(s_comp_linescan_abs)
    s_comp_linescan_zetalist.append(s_comp_linescan_zeta)
    s_comp_linescan_cllist.append(s_comp_linescan_cl)
print('done!')

Sa
!
!!
!!!
Solution converged to less than 0.001 weight percent, after 6 iterations.
!!!!
Sb
!
!!
!!!
Solution converged to less than 0.001 weight percent, after 46 iterations.
!!!!
done!


In [20]:
colors=['r','g','b']
plt.close('all')
for i in range(len(s_comp_linescan_cllist)):
    element=elements[i]
    for k, el in enumerate(element):
        plt.figure(str(i)+'_'+str(el))
        plt.hold(True)
        cz=s_comp_linescan_zetalist[i][k]
        ccl=s_comp_linescan_cllist[i][k]
        cab=s_comp_linescan_abslist[i][k]
        
        scale=s.axes_manager.as_dictionary()['axis-0']['scale']
        x = np.arange(len(cab.data))*scale
        
        plt.plot(x,ccl.data,'b')
        plt.plot(x,cz.data,'g')
        plt.plot(x,cab.data,'r')
        plt.tight_layout()
        if i == 0:
            plt.xlim([None,360*scale])
            if k < 2:
                plt.ylim([-10,55])
            else:
                plt.ylim([-10,110])
        else:
            plt.ylim([-10,110])
        plt.legend(['CL','Zeta','Zeta (abs)'],loc='best')
        plt.grid(axis='y')
        plt.xlabel('X (nm)')
        plt.ylabel('Composition (%)')
        plt.savefig('../PROJECT/thesis/fig/q-new/'+'oldzetas_'+str(snames[i])+'_'+str(lines[i][k]))
    plt.show()

In [26]:
print(zeta)

[[608.0, 706.0, 738.0, 1283.0], [910.66666666666663, 1283.0, 2391.5]]


## Make heatmap Heattreated

In [ ]:
#Heat-treated

zeta = [As,Ga,GeK,Pd]
kfactors = [kAs_k, kGa_k, kGe_k,kPd_m]

elements=[['Ga'],['As'],['Au'],['Pd'],['Ge']]
zeta=[GaK,AsK,AuM,PdL,GeK]
zeta = [np.mean(z) for z in zeta]
kfactors=[kGaK,kAsK,kAuM,kPdL,kGeK]
bw=[bwGeneral, bwGeneral, bwPdLAuM, bwPdLAuM, bwGeneral]
lines=['Ga_Ka','As_Ka','Au_Ma','Pd_La','Ge_Ka']

# Attempt with AU, without GaAs:
# zeta= [Ge,Pd,Au]
# kfactors=[kGe_l,kPd_m,kAu_m]
# elements=['Ge','Pd','Au']
# lines=np.array(['Ge_La','Pd_La','Au_Ma'])
# bw=[BW3,BW_ge,BW5]

plt.close('all')
sD=hs.load('../Data/160506_SC239_X/D_EDS Spectrum Image.dm3') #For D/E
sE=hs.load('../Data/160506_SC239_X/E_EDS Spectrum Image.dm3') #For D/E
# sB=hs.load('../Data/160506_SC239_X/B_EDS Spectrum Image_top.hdf5')
# sC=hs.load('../Data/160506_SC239_X/C_EDS Spectrum Image_behindC.hdf5')
# sF = hs.load('../Data/160506_SC239_X/F_EDS Spectrum Image_bright.hdf5')
# s=[sB,sC,sF]
slist=[sD,sE]
currentE = np.mean((0.1175,0.1169)) #E
currentD = np.mean((0.1175,0.1181)) #D
currents=[currentD,currentE]
# currents = [.1206,np.mean((.1194,.1181)),np.mean((.1169,.1175))]

s_comp_absl=[]
s_comp_zetal=[]
s_comp_cll=[]
scales=[sD.axes_manager.as_dictionary()['axis-0']['scale'],sE.axes_manager.as_dictionary()['axis-0']['scale']]
folder=['D','E']
snames=['D','E']
for i,(s,current) in enumerate(zip(slist,currents)):

    s.set_elements(elements)
    s.set_lines(lines)
    s.mean('x').mean('y').plot(background_windows=bw)
    s.metadata.General.title = snames[i]
    s.set_microscope_parameters(real_time=0.1,beam_current=current)
    s_ints = s.get_lines_intensity(integration_windows=1.2,background_windows=bw,only_lines=lines)

    s_comp_zeta, s_pt = s.quantification(s_ints, 'zeta', zeta)
    s_comp_cl = s.quantification(s_ints, 'CL', kfactors)
    s_comp_abs, s_pt = zc.quantification_zeta(s, s_ints, zeta, convergence_precision=0.00001)
    
    s_comp_absl.append(s_comp_abs)
    s_comp_zetal.append(s_comp_zeta)
    s_comp_cll.append(s_comp_cl)
    
    from os import makedirs
    print(elements)
    plt.ioff()
#     f=open('../PROJECT/thesis/fig/q/B-C-F/without_Au_GeKline-std.txt','w+')
    for a,(s_zeta, s_cl,s_abs,scale) in enumerate(zip(s_comp_zetal, s_comp_cll, s_comp_absl,scales)):
        for i, el in enumerate(elements):
            for s_,tech in zip([s_zeta, s_cl, s_abs],['zeta','CL','zetaAbs']):
    #         for s_,tech in zip([s_zeta, s_cl],['zeta','CL']):
    #             plt.figure()
                data = s_[i].data#-s_abs[i].data
                mean = data.mean()
                std=np.std(data)
    #             data = bin_ndarray(data, new_shape = (data.shape[0]/2,data.shape[1]/2), operation='mean')
    #             plt.imshow(data, cmap='coolwarm',extent=[0,data.shape[0]*scale,0,data.shape[1]*scale])
    #             plt.xlabel('nm')
    #             plt.ylabel('nm')
    #             plt.colorbar(label='Composition (%)')
    #             try:
    #                 makedirs('../PROJECT/thesis/fig/q/'+folder[a]+'_heatedAUHEATMAP/')
    #             except FileExistsError:
    #                 pass
    #             plt.savefig('../PROJECT/thesis/fig/q/'+folder[a]+'_heatedAU/'+str(el)+'_'+str(tech))
                print(s_[i],'\n',folder[a]+','+str(el)+','+str(tech)+', '+str(mean)+'  '+str(std))
#                 f.write(str(mean)+' '+str(std)+'\n')
#     f.close()
